In [1]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA1
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

In [ ]:
class PGP:
    def __init__(self):
        self.sender_key = RSA.generate(1024)
        self.receiver_key = RSA.generate(1024)
    def authenticate(self, message):
        h = SHA1.new(message.encode())
        signature = pkcs1_15.new(self.sender_key).sign(h)
        return {'message':message, "signature":signature}
    def verify(self, signed_message):
        h = SHA1.new(signed_message['message'].encode())
        try:
            pkcs1_15.new(self.sender_key.public_key()).verify(h, signed_message['signature'])
            return True
        except (ValueError, TypeError):
            return False
    def encrypt(self, message):
        session_key = get_random_bytes(16)
        cipher_aes = AES.new(session_key, AES.MODE_CBC)
        ct = cipher_aes.encrypt(pad(message.encode(), AES.block_size))
        cipher_rsa = PKCS1_OAEP.new(self.receiver_key.public_key())
        enc_key = cipher_rsa.encrypt(session_key)
        return {"enc_key":enc_key, 'iv':cipher_aes.iv, 'ct':ct}
    def decrypt(self, package):
        cipher_rsa = PKCS1_OAEP.new(self.receiver_key)
        session_key = cipher_rsa.decrypt(package['enc_key'])
        cipher_aes = AES.new(session_key, AES.MODE_CBC, package['iv'])
        pt = unpad(cipher_aes.decrypt(package['ct']), AES.block_size)
        return pt.decode()

In [3]:
pgp = PGP()
msg = "Hello world"
print("Testing Authentication")
signed = pgp.authenticate(msg)
print("Verified: ", pgp.verify(signed))

Testing Authentication
Verified:  True


In [ ]:
print("\nTesting Confidentiality ")
enc = pgp.encrypt_message(msg)
dec = pgp.decrypt_message(enc)
print(f"Encrypted : {enc}\n Decrypted : {dec}")

print("\n complete pgp")
enc_pkg, sig = pgp.pgp_send(msg)
final_msg, auth_ok = pgp.pgp_received(enc_pkg, sig)
print("Decrypted : ", final_msg)
print("Authentication verified : ", auth_ok)

In [8]:
class PGP_Check:
    def __init__(self):
        self.sender_key = RSA.generate(1024)
        self.receiver_key = RSA.generate(1024)
    def authentication(self, message):
        h = SHA1.new(message.encode())
        signature = pkcs1_15.new(self.sender_key).sign(h)
        return {'message': message, 'signature':signature}
    def verify(self, signed_message):
        h = SHA1.new(signed_message['message'].encode())
        try:
            pkcs1_15.new(self.sender_key.public_key()).verify(h,signed_message['signature'])
            return True
        except (ValueError, TypeError):
            return False
    def encrypt(self, message):
        session_key = get_random_bytes(16)
        cipher_aes = AES.new(session_key, AES.MODE_CBC)
        ct = cipher_aes.encrypt(pad(message.encode(),AES.block_size))
        cipher_rsa = PKCS1_OAEP.new(self.receiver_key.public_key())
        enc_key = cipher_rsa.encrypt(session_key)
        return {"enc_key":enc_key, 'iv':cipher_aes.iv, 'ct':ct}
    def decrypt(self, package):
        cipher_rsa = PKCS1_OAEP.new(self.receiver_key)
        session_key = cipher_rsa.decrypt(package['enc_key'])
        cipher_aes = AES.new(session_key, AES.MODE_CBC, package['iv'])
        pt = unpad(cipher_aes.decrypt(package['ct']),AES.block_size)
        return pt.decode()
    def pgp_send(self, message):
        signed = self.authentication(message)
        full_message = f"{message}|SIG"
        encrypted = self.encrypt(full_message)
        return encrypted, signed['signature']
    def pgp_received(self, encrypted, signature):
        decrypted = self.decrypt(encrypted)
        message = decrypted.split('|SIG')[0]
        return message, self.verify({'message':message, 'signature':signature})

In [9]:
pgp = PGP_Check()
msg = "This is a confidential message for PGP testing"

In [10]:
print("\n🔹 Testing Authentication")
signed = pgp.authentication(msg)
print("Verified:", pgp.verify(signed))


🔹 Testing Authentication
Verified: True


In [11]:
print("\n🔹 Testing Confidentiality")
enc = pgp.encrypt(msg)
dec = pgp.decrypt(enc)
print("Decrypted:", dec)


🔹 Testing Confidentiality
Decrypted: This is a confidential message for PGP testing


In [12]:
print("\n🔹 Testing Complete PGP")
enc_pkg, sig = pgp.pgp_send(msg)
final_msg, auth_ok = pgp.pgp_received(enc_pkg, sig)
print("Decrypted:", final_msg)
print("Authentication Verified:", auth_ok)



🔹 Testing Complete PGP
Decrypted: This is a confidential message for PGP testing
Authentication Verified: True
